# An Example ASDF Design Workflow for CCSPs

## Overview

In this notebook, we'll illustrate one possible method for starting to incorporate ASDF file design into your workflow and pipeline.

We'll assume here that you're starting with a FITS file, though there is no need to do so. We'll extract the WCS from this FITS file's header and convert it into something ASDF-flavored, and then we'll extract the rest of the metadata from the FITS header. (But if you need more a complicated WCS than FITS can easily support, or if you are making alterations to a Roman ASDF file, then you probably *shouldn't* start with a FITS File.)

The rest of the example design process would, in full, go something like this:
- Design an ASDF tree structure and sample content for your product, passing nested dictionaries into an ASDF file object.
- Save the resultant ASDF file object to an ASDF file: your initial sample file (without a corresponding schema).
- Draft a schema that matches the initial sample file you just designed, in consultation with MAST and the RAD maintainers. Send the initial sample file and draft schema to MAST and the RAD maintainers in a Github issue to the RAD repository, which will evolve into a pull request in consultation with MAST and the RAD maintainers. **Note:** We'll largely elide the details of this step in this notebook. For more information on the schema-writing part of the process, see the [File Design Guidelines](https://outerspace.stsci.edu/spaces/DraftMASTCONTRIB/pages/344588706/.File+Design+for+PITs+v1.0#id-.FileDesignforPITsv1.0-Requirements).
- Install the appropriate branch/fork of the RAD and roman_datamodels (with MAST and the RAD's help) into your pipeline environment.
- Pass your ASDF tree into a Roman data model object. Save the result as your revised sample file, which now tags your schema, and deliver it to MAST for validation.

**Note:** this is not the only possible workflow. For example, some people prefer to design the schema first, before constructing the ASDF file object.

## Imports and installations

Later, we'll need a particular branch in a particular fork of the `roman_datamodels` repository, where we've set up an ASDF schema and Roman data model for the product we'll be converting. You should only use this fork in the context of this specific notebook; when you're ready to start working on your own data, install the main branch of [roman_datamodels](https://github.com/spacetelescope/roman_datamodels) proper.

In [1]:
%%capture captured
!pip install git+https://github.com/adrianlucy/roman_datamodels.git@ccsp_schemas_for_notebook

You may need to restart your kernel for that to take effect.

That installation, incidentally, installed everything else that we need as dependencies. So now let's run our imports:

In [2]:
from astropy.io import fits  # For loading the FITS file that we'll convert
import numpy as np  # For array and matrix wrangling

import asdf  # For building the ASDF tree

from astropy.time import Time  # For passing Time objects into the ASDF tree

import gwcs  # For building the WCS
from gwcs import coordinate_frames as cf  # For building the WCS
from astropy.modeling import models  # For building the WCS
from astropy import coordinates as coord  # For building the WCS

import roman_datamodels.datamodels as rdm  # Only used for the final step

## Opening the FITS file

Next, we'll open the FITS file that we want to convert to ASDF. For the purposes of this tutorial, we'll use a simple 2D image file from the [FIMS-SPEAR mission](https://outerspace.stsci.edu/spaces/SPEARFIMS/overview). The main science data array is in the primary HDU, which is accompanied by concomitant images of net photon counts and an exposure map in HDU1 and HDU2, respectively.

In [3]:
# For best practice, in your pipeline this would be `with fits.open("filename"):`
# Get what you need, then close the file.
hdul = fits.open("https://archive.stsci.edu/mccm/fims-spear/spear/vela/mccm_fims-spear_spear-ap100_vela_long-c-iv_v1.0_img.fits")

hdul.info()

Filename: /Users/alucy/.astropy/cache/download/url/4fb09bbd351a92dc11a9029b6feea2d1/contents
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (512, 512)   float32   
  1  COUNT_IMAGE    1 ImageHDU        12   (512, 512)   float32   
  2  EXPOSURE_MAP    1 ImageHDU        13   (512, 512)   float32   


Let's take a look at the headers for each HDU. We see that most of the metadata is in the primary header, and that the concomitant images share the primary array's World Coordinate System (WCS). The WCS maps the array pixel coordinates to world coordinates like wavelength, time, or (in this case) sky coordinates.

In [4]:
header0 = hdul[0].header
header0

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  512                                                  
NAXIS2  =                  512                                                  
EXTEND  =                    T                                                  
DATE    = '2023-04-12'         / Creation date (CCYY-MM-DD) of FITS header      
BUNIT   = 'photon/(cm2*s*sr)'  / Physical unit of the array values              
PC1_1   =                  1.0 / Transformation matrix                          
PC2_2   =                  1.0 / Transformation matrix                          
CTYPE1  = 'RA---TAN'           / X-axis                                         
CRVAL1  =                129.0 / Origin coordinate                              
CRPIX1  =                256

In [5]:
hdul[1].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  512                                                  
NAXIS2  =                  512                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
EXTNAME = 'COUNT_IMAGE'        / extension name                                 
BUNIT   = 'photon  '           / Physical unit of the array values [Count/pixel]
CHECKSUM= '9mCKEkBH9kBHEkBH'   / HDU checksum updated 2023-04-12T17:22:30       
DATASUM = '328920997'          / data unit checksum updated 2023-04-12T17:22:30 
COMMENT Continuum-subtracted photon count map                                   

In [6]:
hdul[2].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  512                                                  
NAXIS2  =                  512                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
EXTNAME = 'EXPOSURE_MAP'       / extension name                                 
BUNIT   = 's       '           / Physical unit of the array values [s/pixel]    
CHECKSUM= '4p284n164n164n16'   / HDU checksum updated 2023-04-12T17:22:30       
DATASUM = '738137009'          / data unit checksum updated 2023-04-12T17:22:30 
COMMENT Exposure time map. Array values correspond to net exposure time in pixel
COMMENT .                   

## Converting FITS WCS to GWCS

Next, we'll use `gwcs.utils.make_fitswcs_transform` to convert the FITS WCS keywords into a `GWCS` transform. `GWCS` is ASDF's solution for storing complex WCS solutions with distortions and/or nonlinear components. In this case, the WCS is a simple gnomonic ("TAN") projection, but we can get a feel for how ASDF thinks about WCS.

In [7]:
# Extract the transform from the FITS header
transform = gwcs.utils.make_fitswcs_transform(header0)

`make_fitswcs_transform` doesn't identify the input and output frame, so we'll need to specify those manually. Looking at the FITS header, we see that the output world coordinates are in ICRS.

In [8]:
# Define the frames
pixel_frame = cf.Frame2D(axes_names=('x','y'), name='pixel')  # Input pixel frame
sky_frame = cf.CelestialFrame(reference_frame=coord.ICRS(),
                              axes_names=('ra','dec'),
                              name='icrs',
                              axis_physical_types = ('pos.eq.ra', 'pos.eq.dec'))  # Use the UCDs for RA/Dec in the axis_physical_types

Finally, we'll pass the transform and the input/output frames into a `GWCS` WCS object.

In [9]:
wcs = gwcs.WCS(forward_transform=transform, input_frame=pixel_frame, output_frame=sky_frame)

Let's take a look at that WCS object. We see that it is comprised of a single (multi-component) transform step from pixel to ICRS coordinates.

In [10]:
print(wcs)

 From   Transform  
----- -------------
pixel CompoundModel
 icrs          None


In [11]:
wcs

<WCS(output_frame=icrs, input_frame=pixel, forward_transform=Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('alpha_C', 'delta_C')
Model set size: 1
Expression: [0] & [1] | [2] | [3] & [4] | [5] | [6]
Components: 
    [0]: <Shift(offset=-255.5, name='crpix1')>

    [1]: <Shift(offset=-255.5, name='crpix2')>

    [2]: <AffineTransformation2D(matrix=[[1., 0.], [0., 1.]], translation=[0., 0.], name='pc_matrix')>

    [3]: <Scale(factor=-0.0244141, name='cdelt1')>

    [4]: <Scale(factor=0.0244141, name='cdelt2')>

    [5]: <Pix2Sky_Gnomonic()>

    [6]: <RotateNative2Celestial(lon=129., lat=-46.25, lon_pole=180., name='crval')>
Parameters:
    offset_0 offset_1  matrix_2  translation_2 ...  factor_4 lon_6 lat_6  lon_pole_6
    -------- -------- ---------- ------------- ... --------- ----- ------ ----------
      -255.5   -255.5 1.0 .. 1.0    0.0 .. 0.0 ... 0.0244141 129.0 -46.25      180.0)>

Note FITS pixel indexing is 1-based while ASDF/Python indexing is 0-based, so the `CRVALi` sky coordinates are offset by 1 pixel from the `CRPIXj` pixel coordinates in the FITS header:

In [12]:
print(header0['CRVAL1'], header0['CRVAL2'])

print(wcs.pixel_to_world(header0['CRPIX1']-1, header0['CRPIX2']-1))

129.0 -46.25
<SkyCoord (ICRS): (ra, dec) in deg
    (129., -46.25)>


Just to double-check the work of `make_fitswcs_transform`, we can also try following the "[FITS Equivalent WCS Example](https://gwcs.readthedocs.io/en/latest/gwcs/fits_analog.html)" workflow from the GWCS documentation. We'll make two small changes to that example: drop the use of astropy units, and (for precision in nomenclature when `CDELTi != 1`) drop `rotation.input_units_equivalencies` in favor of a `models.Scale` component.

In [13]:
# Adapted directly from https://gwcs.readthedocs.io/en/latest/gwcs/fits_analog.html

# Adjust by 1 to accomodate the different indexing
shift_by_crpix = models.Shift(-(header0['CRPIX1'] - 1)) & models.Shift(-(header0['CRPIX2'] - 1))

# Set the PCi_j matrix
try:
    matrix = np.array([[header0['PC1_1'], header0['PC1_2']],
                       [header0['PC2_1'], header0['PC2_2']]])
except KeyError:  # If PC1_2, PC2_1 are not present
    matrix = np.array([[header0['PC1_1'], 0.0],
                       [0.0, header0['PC2_2']]])

# Apply the pixel scale
scale = (models.Scale(header0['CDELT1']) & models.Scale(header0['CDELT2']))

# Apply the PCi_j matrix to rotate the inputs
rotation = models.AffineTransformation2D(matrix, translation=[0, 0])
rotation.inverse = models.AffineTransformation2D(np.linalg.inv(matrix), translation=[0, 0])

# Apply a gnomonic projection
tan = models.Pix2Sky_TAN()

# Apply a rotation on the sky using CRVAL.
celestial_rotation =  models.RotateNative2Celestial(header0['CRVAL1'], header0['CRVAL2'], 180)

# Combine all the steps into a single transform
det2sky = shift_by_crpix | rotation | scale | tan | celestial_rotation
det2sky.name = "linear_transform"

# Define the frames as before
pixel_frame = cf.Frame2D(axes_names=('x','y'), name='pixel')  # Input pixel frame
sky_frame = cf.CelestialFrame(reference_frame=coord.ICRS(),
                              axes_names=('ra','dec'),
                              name='icrs',
                              axis_physical_types = ('pos.eq.ra', 'pos.eq.dec'))  # Use the UCDs for RA/Dec in the axis_physical_types

# Put the transform and the frames together
pipeline = [(pixel_frame, det2sky), (sky_frame, None)]

# Pass them into a gwcs WCS object
wcs_manual = gwcs.WCS(pipeline)

In [14]:
print(wcs_manual)

 From    Transform    
----- ----------------
pixel linear_transform
 icrs             None


In [15]:
wcs_manual

<WCS(output_frame=icrs, input_frame=pixel, forward_transform=Model: CompoundModel
Name: linear_transform
Inputs: ('x0', 'x1')
Outputs: ('alpha_C', 'delta_C')
Model set size: 1
Expression: [0] & [1] | [2] | [3] & [4] | [5] | [6]
Components: 
    [0]: <Shift(offset=-255.5)>

    [1]: <Shift(offset=-255.5)>

    [2]: <AffineTransformation2D(matrix=[[1., 0.], [0., 1.]], translation=[0., 0.])>

    [3]: <Scale(factor=-0.0244141)>

    [4]: <Scale(factor=0.0244141)>

    [5]: <Pix2Sky_Gnomonic()>

    [6]: <RotateNative2Celestial(lon=129., lat=-46.25, lon_pole=180.)>
Parameters:
    offset_0 offset_1  matrix_2  translation_2 ...  factor_4 lon_6 lat_6  lon_pole_6
    -------- -------- ---------- ------------- ... --------- ----- ------ ----------
      -255.5   -255.5 1.0 .. 1.0    0.0 .. 0.0 ... 0.0244141 129.0 -46.25      180.0)>

That looks like the same WCS object! Let's double-check over an arbitrary range of pixels:

In [16]:
for x in range (-20,-20):
    for y in range(-20,20):
        test = np.asarray(wcs(x,y)) - np.asarray(wcs_manual(x,y))
        if test.any() != 0:
            print("The WCS's don't match")

If you're having trouble constructing a WCS for your data, your MAST and SOC contacts are happy to help, and can consult with the `gwcs` developers on your behalf as needed.

## Assembling metadata dictionaries

Next, we'll pass a variety of metadata (including the WCS object we created above) from the FITS header into Python dictionaries, before we construct our ASDF file object.

Let's take a look at the [archival common metadata table in the expandable linked here](https://outerspace.stsci.edu/spaces/DraftMASTCONTRIB/pages/344588706/.File+Design+for+PITs+v1.0#id-.FileDesignforPITsv1.0-pits_common) (or if you prefer to look at schemas, see [ccsp_minimal](https://rad--747.org.readthedocs.build/en/747/generated/schemas/CCSP/ccsp_minimal-1.0.0.html) and [ccsp_custom_product](https://rad--747.org.readthedocs.build/en/747/generated/schemas/CCSP/ccsp_custom_product-1.0.0.html)) and start filling out what we can for those keys, supplemented by the other, unique metadata found in this FITS header.

ASDF data and metadata can be nested in a hierarchical tree structure, collecting related information together. So we'll start with small dictionaries, and then pass those dictionaries into a top-level dictionary in a nested structure.

In [17]:
ccsp = {
    "name": "SPEAR",
    "investigator": "Jerry Edelstein",
    "archive_lead": "Martin Sirk",
    "doi": header0['DOI'],
    "file_version": header0['VER'],
    "data_release_id": "DR1",
    "license": header0['LICENSE'],
    "license_url": header0['LICENURL'],
    "target_name": header0['TARG'],
    "intent": "SCIENCE",
    "target_keywords": "Supernova remnants",  # Chosen from https://astrothesaurus.org/concept-select/
    "target_keywords_id": 1667,  # From the https://astrothesaurus.org/concept-select/ entry above
}

instrument = {
    "name": header0['INSTRUME'],
    "detector": None,
    "optical_element": header0['FILTER'],
}

target_coordinates = {
    "reference_frame": header0['RADESYS'],
    "ra": header0['RA_TARG'],
    "dec": header0['DEC_TARG'],
}

wavelength = {
    "band": "UV",
    "minimum": float(header0['LAM-MIN']),
    "maximum": float(header0['LAM-MAX']),
}

To populate the start and end date-times of this product, we'll convert the values from the FITS header into astropy Time objects before passing them into the dictionary, for compatibility with the ASDF Time schema.

And the exposure time in the FITS header that best matches the mandatory `exposure_time` sub-key's definition is `EXP-SLIT`, so we'll use that as the characteristic exposure time under `exposure`.

In [18]:
start = Time(header0['DATE-BEG'], format='isot', scale=header0['TIMESYS'].lower())
end = Time(header0['DATE-END'], format='isot', scale=header0['TIMESYS'].lower())

exposure = {
    "start_time": start,
    "end_time": end,
    "exposure_time": header0['EXP-SLIT'],
}

We have a lot of detail on the statistics of the exposure map, beyond the scope of the `exposure` parent key defined in [ccsp_custom_product](https://rad--747.org.readthedocs.build/en/747/generated/schemas/CCSP/ccsp_custom_product-1.0.0.html), so we'll create an `exposure_stats` tree to group this information.

Because there is no character count limit on ASDF keywords, we can be slightly more verbose than we would be in FITS (e.g., `pixel_exposure_time` instead of `EXP-PIX`). Still, the definition of these keys is left to the schema.

In [19]:
exposure_stats = {
    "median_exposure_time": header0['EXP-SLIT'],
    "max_exposure_time": header0['EXP-SMAX'],
    "min_exposure_time": header0['EXP-SMIN'],
    "pixel_exposure_time": header0['EXP-PIX'],
}

We'll also use our WCS object to get the sky coordinate boundaries of the image, to pass into an `s_region` keyword that will make this product discoverable in coordinate cone searches upon ingestion into MAST.

In [20]:
# Assumes a fully-illuminated 2D image
def s_region_fullchip(wcs, data):
    s_region_parts = ['POLYGON', 'ICRS']
    naxis1, naxis2 = data.shape  # Get pixel image shape

    s_region_parts.extend([
        str(wcs(0,0)[0]),  # RA of 1st vertex
        str(wcs(0,0)[1]),  # Dec of 1st vertex
        str(wcs(naxis1-1, 0)[0]),  # RA of 2nd vertex
        str(wcs(naxis1-1, 0)[1]),  # Dec of 2nd vertex
        str(wcs(naxis1-1, naxis2-1)[0]),  # RA of 3rd vertex
        str(wcs(naxis1-1, naxis2-1)[1]),  # Dec of 3rd vertex
        str(wcs(0, naxis2-1)[0]),  # RA of 4th vertex
        str(wcs(0, naxis2-1)[1])  # Dec of 4th vertex
    ])

    s_region = " ".join(s_region_parts)

    return s_region

s_region = s_region_fullchip(wcs, hdul[0].data)

print(s_region)

POLYGON ICRS 139.072956118736 -52.03248109358212 118.92704388126397 -52.03248109358211 120.95392087678839 -39.757741620879415 137.04607912321157 -39.757741620879415


## Assembling the ASDF tree

Now that we've deconstructed the FITS file and mapped much of its contents to Python dictionaries, we can construct the ASDF tree in a readable way. 

In [21]:
# Make the ASDF tree
af = asdf.AsdfFile({
    "meta": {
        "wcs": wcs,
        "ccsp": ccsp,
        "exposure": exposure,
        "exposure_stats": exposure_stats,
        "instrument": instrument,
        "telescope": header0['TELESCOP'],
        "target_coordinates": target_coordinates,
        "s_region": s_region,
        "pixel_scale": header0['CDELT1']/3600.,  # Convert deg to arcsec
        "wavelength": wavelength,
        "aperture": header0['APERTURE'],
        "grasp": header0['GRASP']
        
    },
    "data": hdul[0].data,
    "count": hdul[1].data,
    "exp": hdul[2].data,
})

Now we can write that ASDF file object to an ASDF file on-disk:

In [22]:
af.write_to("sample_file.asdf")

At this point, you would start writing a data product schema compatible with this sample, following [the guidelines](https://outerspace.stsci.edu/spaces/DraftMASTCONTRIB/pages/344588706/.File+Design+for+PITs+v1.0#id-.FileDesignforPITsv1.0-asdfASDFFileDesign). Then, you would share this file and its corresponding schema with MAST and the RAD maintainers in a [Github issue](github.com/spacetelescope/rad/issues/new) to the RAD repository.

Taking a look at this sample file, we see that it's pretty good, but many of the keys aren't clearly defined:

In [23]:
testaf_sample = asdf.open('sample_file.asdf')
testaf_sample.info(max_rows=None, max_cols=None)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 5.0.0
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.6.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_standard
│   │ │ └─version (str): 1.4.0
│   │ └─software (Software)
│   │   ├─name (str): asdf
│   │   └─version (str): 5.0.0
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.4.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_wcs_schemas
│   │ │ └─version (str): 0.5.0
│   │ └─software (Software)
│   │   ├─name (str): gwcs
│   │   └─version (str): 0.26.0
│   ├─[2] (Extensi

That's because key definitions and units in ASDF are externalized to the schemas, and we haven't yet linked this file to a schema:

In [24]:
print(af.schema_info("description","roman.data"))

None


## Converting to a DataModel and tagging the schema

Now that we have the file contents mapped to ASDF, you would work on making a DataModel for the data. This would involve:
- defining a schema
- registering this schema using roman_datamodels

Since the above changes are implemented by modifying `rad` and `roman_datamodels`, and these examples haven't been merged into a release, the following code will only work by installing the fork at `git+https://github.com/adrianlucy/roman_datamodels.git@ccsp_schemas_for_notebook`, which we did at the top of this notebook.

First let's look at the added schema by providing its URI and asking asdf to load the resource.

In [25]:
resource = asdf.get_config().resource_manager["asdf://stsci.edu/datamodels/roman/schemas/CCSP/EXAMPLE/example_spear_pointed_image-1.0.0"]
print(resource.decode("ascii"))

%YAML 1.1
---
$schema: asdf://stsci.edu/datamodels/roman/schemas/rad_schema-1.0.0
id: asdf://stsci.edu/datamodels/roman/schemas/CCSP/EXAMPLE/example_spear_pointed_image-1.0.0

title: Example SPEAR pointed image CCSP product

datamodel_name: ExampleSpearPointedImageModel

type: object
properties:
  meta:
    allOf:
      - $ref: asdf://stsci.edu/datamodels/roman/schemas/CCSP/ccsp_custom_product-1.0.0
      - required:
          [
            exposure,
            instrument,
            telescope,
            target_coordinates,
            s_region,
            pixel_scale,
            wavelength,
          ]
      - type: object
        properties:
          wcs:
            title: World Coordinate System (WCS)
            description: |
              WCS for the data array and all concomitant arrays
            tag: tag:stsci.edu:gwcs/wcs-*
          aperture:
            title: Slit shutter aperture mode
            type: string
          exposure_stats:
            title: Exposure 

In the above schema note that:
- the common `ccsp_custom_product-1.0.0` schema is **referenced**
- the ASDF structure is described (and constrained) by the schema (for example, `data` must be a 2-dimensional image with float32 values and units of photons/cm^2/s/sr.

The addition of this schema to RAD and a small modification to roman_datamodels allows us to use this schema for a new DataModel `ExampleSpearPointedImageModel`. Let's create a new instance of that model with the tree we constructed above.

In [26]:
model = rdm.ExampleSpearPointedImageModel.create_from_model(af.tree)

model.info(max_rows=None, max_cols=None)

root (AsdfObject)
└─roman (ExampleSpearPointedImage) # Example SPEAR pointed image CCSP product
  ├─meta (dict)
  │ ├─wcs (WCS) # World Coordinate System (WCS)
  │ ├─ccsp (dict) # Community Contributed Science Product (CCSP) information
  │ │ ├─name (str): SPEAR # CCSP name
  │ │ ├─investigator (str): Jerry Edelstein # CCSP Principal Investigator
  │ │ ├─archive_lead (str): Martin Sirk # CCSP staff lead for MAST ingest
  │ │ ├─doi (str): doi:10.17909/dsbe-kj54 # CCSP Digital Object Identifier
  │ │ ├─file_version (str): 1.0 # Version of this file
  │ │ ├─data_release_id (str): DR1 # CCSP collection data release
  │ │ ├─license (str): CC BY 4.0 # License for use of these data.
  │ │ ├─license_url (str): https://creativecommons.org/licenses/by/4.0/ # URL of license for use of these data.
  │ │ ├─target_name (str): Vela # Target name
  │ │ ├─intent (str): SCIENCE # Observation intent
  │ │ ├─target_keywords (str): Supernova remnants # UAT keywords
  │ │ └─target_keywords_id (int): 1667 # 

Let's try validating the model we've created against its schema:

In [27]:
try:
   model.validate()
except asdf.exceptions.ValidationError as err:
    print(f"ValidationError({err.message})")

ValidationError('file_date' is a required property)


Oops! We've forgotten to add the required `file_date` key. Let's do that, and try again:

In [28]:
model["meta"]["file_date"] = Time(Time.now(), format='isot')

In [29]:
try:
   model.validate()
except asdf.exceptions.ValidationError as err:
    print(f"ValidationError({err.message})")

Now that our model is valid we can save it to an ASDF file:

In [30]:
model.save("ccsp_example_spear_vela_long-c-iv_v0.0_img.asdf")

PosixPath('ccsp_example_spear_vela_long-c-iv_v0.0_img.asdf')

## Examining the results

We're done, but let's take some time to look at what we've created. If we open that file up with `roman_datamodels`, we see that the Roman key is tagged with our schema, and the keys are now commented with their titles:

In [31]:
testdm = rdm.open('ccsp_example_spear_vela_long-c-iv_v0.0_img.asdf')

In [32]:
testdm.info(max_rows=None, max_cols=None)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 5.0.0
├─history (AsdfDictNode)
│ └─extensions (AsdfListNode)
│   ├─0 (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.6.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_standard
│   │ │ └─version (str): 1.4.0
│   │ └─software (Software)
│   │   ├─name (str): asdf
│   │   └─version (str): 5.0.0
│   ├─1 (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.4.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_wcs_schemas
│   │ │ └─version (str): 0.5.0
│   │ └─software (Software)
│   │   ├─name (str): gwcs
│   │   └─version (str): 0.26.0
│   ├─

And we can also retrieve the description and units of these keys programatically:

In [33]:
print(testdm.schema_info("description","roman.meta.target_coordinates.ra"))
print(testdm.schema_info("unit","roman.meta.target_coordinates.ra"))

{'description': Characteristic right ascension in degrees; typically the
center of the spatial image, or the target coordinates
of a spectrum or light curve.
}
{'unit': deg}


And because the right branch of `roman_datamodels` is installed into our Python environment, the schema is also similarly registered by the ASDF package while we're in this environment:

In [34]:
testaf = asdf.open('ccsp_example_spear_vela_long-c-iv_v0.0_img.asdf')

In [35]:
testaf.info(max_rows=None, max_cols=None)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 5.0.0
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.6.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_standard
│   │ │ └─version (str): 1.4.0
│   │ └─software (Software)
│   │   ├─name (str): asdf
│   │   └─version (str): 5.0.0
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/gwcs-1.4.0
│   │ ├─manifest_software (Software)
│   │ │ ├─name (str): asdf_wcs_schemas
│   │ │ └─version (str): 0.5.0
│   │ └─software (Software)
│   │   ├─name (str): gwcs
│   │   └─version (str): 0.26.0
│   ├─[2] (Extensi

In [36]:
# This only works because `roman_datamodels` is installed in our environment,
# serving to connect the file to its tagged schema
print(testaf.schema_info("description","roman.meta.target_coordinates.ra"))
print(testaf.schema_info("unit","roman.meta.target_coordinates.ra"))

{'description': Characteristic right ascension in degrees; typically the
center of the spatial image, or the target coordinates
of a spectrum or light curve.
}
{'unit': deg}


In [37]:
# Let's finally close the FITS file, now that we're done with it
hdul.close()